# **Labs 1 and 2 PySpark:**

In these labs we will be using the "[[NeurIPS 2020] Data Science for COVID-19 (DS4C)](https://www.kaggle.com/datasets/kimjihoo/coronavirusdataset?select=PatientInfo.csv)" dataset, retrieved from [Kaggle](https://www.kaggle.com/) on 1/6/2022, for educational non commercial purpose, License
[CC BY-NC-SA 4.0
](https://creativecommons.org/licenses/by-nc-sa/4.0/)


The csv file that we will be using in this lab is **PatientInfo**.

## PatientInfo.csv

**patient_id**
the ID of the patient

**sex**
the sex of the patient

**age**
the age of the patient

**country**
the country of the patient

**province**
the province of the patient

**city**
the city of the patient

**infection_case**
the case of infection

**infected_by**
the ID of who infected the patient


**contact_number**
the number of contacts with people

**symptom_onset_date**
the date of symptom onset

**confirmed_date**
the date of being confirmed

**released_date**
the date of being released

**deceased_date**
the date of being deceased

**state**
isolated / released / deceased

### Import the pyspark and check it's version

In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=a6db3cf11a19bbef7cdbaffe304cd3f9e360a1fb0abf643e424d61ab3bf19855
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


### Import and create SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('RDD').getOrCreate()

sc = spark.sparkContext

In [ ]:
spark

### Load the PatientInfo.csv file and show the first 5 rows

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
df = spark.read.csv('/content/PatientInfo.csv',header=True,inferSchema=True)


In [ ]:
df.show(5,truncate=False)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|sex   |age|country|province|city       |infection_case      |infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state   |
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|male  |50s|Korea  |Seoul   |Gangseo-gu |overseas inflow     |null       |75            |2020-01-22        |2020-01-23    |2020-02-05   |null         |released|
|1000000002|male  |30s|Korea  |Seoul   |Jungnang-gu|overseas inflow     |null       |31            |null              |2020-01-30    |2020-03-02   |null         |released|
|1000000003|male  |50s|Korea  |Seoul   |Jongno-gu  |contact with patient|2002000001 |17            |null              |2020-01-30    |2020-0

### Display the schema of the dataset

In [ ]:
df.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)



### Display the statistical summary

In [ ]:
df_summary = df.summary()
df_summary.show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|    5165|
|   mean|2.8636345618679576E9|  null|null|      null|    null|          null|                null|2.2845944015643125E9|1.6772572523506988E7|              null|    null|
| stddev| 2.074210725277473E9|  null|null|      null|    null|          null|                null|1.5265072953383324E9| 3.093097580985502E8|              n

### Using the state column.
### How many people survived (released), and how many didn't survive (isolated/deceased)?

In [ ]:
df.select('state').groupBy('state').count().show()

+--------+-----+
|   state|count|
+--------+-----+
|isolated| 2158|
|released| 2929|
|deceased|   78|
+--------+-----+



In [ ]:
df_count = spark.sql('select state ,count(state) from  ')

### Display the number of null values in each column

In [ ]:
col =df.columns
col

['patient_id',
 'sex',
 'age',
 'country',
 'province',
 'city',
 'infection_case',
 'infected_by',
 'contact_number',
 'symptom_onset_date',
 'confirmed_date',
 'released_date',
 'deceased_date',
 'state']

In [ ]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [ ]:
df.select('state').where(col('state').isNull()).count()

0

In [ ]:
df_null = df.select([sum(col(column).isNull().cast("int")).alias(column) for column in df.columns])
df_null.show()

+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         5099|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



## Data preprocessing

### Fill the nulls in the deceased_date with the released_date.
- You can use <b>coalesce</b> function

In [ ]:
df_fill = df.withColumn("deceased_date", coalesce(col("deceased_date"), col("released_date")))

df_fill.select('deceased_date','released_date').show()

+-------------+-------------+
|deceased_date|released_date|
+-------------+-------------+
|   2020-02-05|   2020-02-05|
|   2020-03-02|   2020-03-02|
|   2020-02-19|   2020-02-19|
|   2020-02-15|   2020-02-15|
|   2020-02-24|   2020-02-24|
|   2020-02-19|   2020-02-19|
|   2020-02-10|   2020-02-10|
|   2020-02-24|   2020-02-24|
|   2020-02-21|   2020-02-21|
|   2020-02-29|   2020-02-29|
|   2020-02-29|   2020-02-29|
|   2020-02-27|   2020-02-27|
|         null|         null|
|   2020-03-12|   2020-03-12|
|         null|         null|
|   2020-03-11|   2020-03-11|
|   2020-03-01|   2020-03-01|
|         null|         null|
|   2020-03-08|   2020-03-08|
|         null|         null|
+-------------+-------------+
only showing top 20 rows



### Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
- <b> Hint: You need to typecast these columns as date first <b>

In [ ]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.types import *

In [ ]:
from pyspark.sql.functions import col, coalesce, to_date

In [ ]:
df_diff = df_fill.withColumn("no_days",to_date('deceased_date')- to_date('confirmed_date'))

In [ ]:
df_diff.select('deceased_date','confirmed_date','no_days').show(5)

+-------------+--------------+-----------------+
|deceased_date|confirmed_date|          no_days|
+-------------+--------------+-----------------+
|   2020-02-05|    2020-01-23|INTERVAL '13' DAY|
|   2020-03-02|    2020-01-30|INTERVAL '32' DAY|
|   2020-02-19|    2020-01-30|INTERVAL '20' DAY|
|   2020-02-15|    2020-01-30|INTERVAL '16' DAY|
|   2020-02-24|    2020-01-31|INTERVAL '24' DAY|
+-------------+--------------+-----------------+
only showing top 5 rows



In [ ]:
df_diff2 = df_fill.withColumn("no_days", datediff(col("deceased_date"), col("confirmed_date")))
df_diff2.select('deceased_date','confirmed_date','no_days').show(5)

+-------------+--------------+-------+
|deceased_date|confirmed_date|no_days|
+-------------+--------------+-------+
|   2020-02-05|    2020-01-23|     13|
|   2020-03-02|    2020-01-30|     32|
|   2020-02-19|    2020-01-30|     20|
|   2020-02-15|    2020-01-30|     16|
|   2020-02-24|    2020-01-31|     24|
+-------------+--------------+-------+
only showing top 5 rows



### Add a is_male column if male then it should yield true, else then False

In [ ]:
df_diff2.select('sex').show(5)

+------+
|   sex|
+------+
|  male|
|  male|
|  male|
|  male|
|female|
+------+
only showing top 5 rows



In [ ]:
df_male = df_diff2.withColumn("is_mal", col('sex') =='male')
df_male.select('sex','is_mal').show(5)

+------+------+
|   sex|is_mal|
+------+------+
|  male|  true|
|  male|  true|
|  male|  true|
|  male|  true|
|female| false|
+------+------+
only showing top 5 rows



In [ ]:
df_male.select('is_mal').where(col('is_mal').isNull()).count()

1122

### Add a is_dead column if patient state is not released then it should yield true, else then False

- Use <b>UDF</b> to perform this task.
- However, UDF is not recommended there is no built in function can do the required operation.
- UDF is slower than built in functions.

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType

In [ ]:
def is_dead(state):
    if state != "released":
        return True
    else:
        return False

is_deadUDF = udf(is_dead, BooleanType())


In [ ]:
df_is_dead = df_male.withColumn("is_dead", is_deadUDF(col("state")))

df_is_dead.select('state','is_dead').show(20)

+--------+-------+
|   state|is_dead|
+--------+-------+
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|deceased|   true|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
|released|  false|
+--------+-------+
only showing top 20 rows



### Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20

In [ ]:
df_is_dead.select('age').show(5)

+---+
|age|
+---+
|50s|
|30s|
|50s|
|20s|
|20s|
+---+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import col, substring, cast, regexp_replace,regexp_extract

In [ ]:
df_is_dead.select('age').where(col('age').isNull()).count()

1380

In [ ]:
df_age = df_is_dead.withColumn("age1", (substring(col("age"), 0, 2).cast("integer")))

print(df_age.select('age1').where(col('age1').isNull()).count())
df_age.select('age','age1').show(5)

1446
+---+----+
|age|age1|
+---+----+
|50s|  50|
|30s|  30|
|50s|  50|
|20s|  20|
|20s|  20|
+---+----+
only showing top 5 rows



In [ ]:
df_age = df_is_dead.withColumn("age", (regexp_replace(col("age"), 's', '').cast("integer")))
print(df_age.select('age').where(col('age').isNull()).count())
df_age.select('age','age').show(5)

1380
+---+---+
|age|age|
+---+---+
| 50| 50|
| 30| 30|
| 50| 50|
| 20| 20|
| 20| 20|
+---+---+
only showing top 5 rows



In [ ]:
df_age.select('age').where(col('age').isNull()).count()

1380

In [ ]:
df_age.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: date (nullable = true)
 |-- released_date: date (nullable = true)
 |-- deceased_date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- no_days: integer (nullable = true)
 |-- is_mal: boolean (nullable = true)
 |-- is_dead: boolean (nullable = true)



### Change age, and no_days  to be typecasted as Double

In [ ]:
df_double_age = df_age.withColumn('age', col('age').cast('double'))
df_double = df_double_age.withColumn('no_days', col('no_days').cast('double'))

### Drop the columns
["patient_id","sex","infected_by","contact_number","released_date","state",
"symptom_onset_date","confirmed_date","deceased_date","country","no_days",
"city","infection_case"]

In [ ]:
dropcol = ["patient_id","sex","infected_by","contact_number","released_date","state", "symptom_onset_date",
                   "confirmed_date","deceased_date","country","no_days", "city","infection_case"]
df_dropped = df_double.drop(*dropcol)


In [ ]:
df_dropped.printSchema()

root
 |-- age: double (nullable = true)
 |-- province: string (nullable = true)
 |-- is_mal: boolean (nullable = true)
 |-- is_dead: boolean (nullable = true)



In [ ]:
df_null1 = df_dropped.select([sum(col(column).isNull().cast("int")).alias(column) for column in df_dropped.columns])
df_null1.show()

+----+--------+------+-------+
| age|province|is_mal|is_dead|
+----+--------+------+-------+
|1380|       0|  1122|      0|
+----+--------+------+-------+



In [ ]:
df_drop = df_dropped.na.drop(subset=["is_mal"])

### Recount the number of nulls now

In [ ]:
df_null2 = df_drop.select([sum(col(column).isNull().cast("int")).alias(column) for column in df_dropped.columns])
df_null2.show()

+---+--------+------+-------+
|age|province|is_mal|is_dead|
+---+--------+------+-------+
|261|       0|     0|      0|
+---+--------+------+-------+



## Now do the same but using SQL select statement

### From the original Patient DataFrame, Create a temporary view (table).

In [ ]:
df.createOrReplaceTempView("view_table")

### Use SELECT statement to select all columns from the dataframe and show the output.

In [ ]:
select_all = spark.sql("select * from view_table")

select_all.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

### *Using SQL commands*, limit the output to only 5 rows

In [ ]:
select_limit = spark.sql("select * from view_table LIMIT 5")

select_limit.show()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

### Select the count of males and females in the dataset

In [ ]:
select_count = spark.sql("select sex, count(sex) AS count from view_table GROUP BY sex")

select_count.show()

+------+-----+
|   sex|count|
+------+-----+
|  null|    0|
|female| 2218|
|  male| 1825|
+------+-----+



In [ ]:
select_count = spark.sql("select sex, count(*) AS count from view_table GROUP BY sex")

select_count.show()

+------+-----+
|   sex|count|
+------+-----+
|  null| 1122|
|female| 2218|
|  male| 1825|
+------+-----+



### How many people did survive, and how many didn't?

In [ ]:
select_survive = spark.sql("select state, count(*) AS count from view_table GROUP BY state")

select_survive.show()

+--------+-----+
|   state|count|
+--------+-----+
|isolated| 2158|
|released| 2929|
|deceased|   78|
+--------+-----+



In [ ]:
select_survive2 = spark.sql("""
    select
    CASE WHEN state = 'released' THEN 'Survived' ELSE 'Not Survived' END AS survive,
        COUNT(*) AS count
    from view_table
    GROUP BY survive
""")

select_survive2.show()

+------------+-----+
|     survive|count|
+------------+-----+
|Not Survived| 2236|
|    Survived| 2929|
+------------+-----+



### Now, let's perform some preprocessing using SQL:
1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [ ]:
df_age1 = df.withColumn("age", (substring(col("age"), 0, 2).cast("integer")))

df_age.select('age').show(5)

+---+
|age|
+---+
| 50|
| 30|
| 50|
| 20|
| 20|
+---+
only showing top 5 rows



In [ ]:
df_select = df_age1.select('sex', 'age', 'province', 'state')
df_select.show(5)

+------+---+--------+--------+
|   sex|age|province|   state|
+------+---+--------+--------+
|  male| 50|   Seoul|released|
|  male| 30|   Seoul|released|
|  male| 50|   Seoul|released|
|  male| 20|   Seoul|released|
|female| 20|   Seoul|released|
+------+---+--------+--------+
only showing top 5 rows



## Machine Learning
### Create a pipeline model to predict is_dead and evaluate the performance.
- Use <b>StringIndexer</b> to transform <b>string</b> data type to indices.
- Use <b>OneHotEncoder</b> to deal with categorical values.
- Use <b>Imputer</b> to fill missing data with mean.

In [ ]:
df_drop.printSchema()

root
 |-- age: double (nullable = true)
 |-- province: string (nullable = true)
 |-- is_mal: boolean (nullable = true)
 |-- is_dead: boolean (nullable = true)



In [ ]:
df_drop.show(5)

+----+--------+------+-------+
| age|province|is_mal|is_dead|
+----+--------+------+-------+
|50.0|   Seoul|  true|  false|
|30.0|   Seoul|  true|  false|
|50.0|   Seoul|  true|  false|
|20.0|   Seoul|  true|  false|
|20.0|   Seoul| false|  false|
+----+--------+------+-------+
only showing top 5 rows



In [ ]:
df_new = df_drop.withColumn('is_mal', col('is_mal').cast('int'))
df_new = df_new.withColumn('is_dead', col('is_dead').cast('int'))
df_new.show(5)

+----+--------+------+-------+
| age|province|is_mal|is_dead|
+----+--------+------+-------+
|50.0|   Seoul|     1|      0|
|30.0|   Seoul|     1|      0|
|50.0|   Seoul|     1|      0|
|20.0|   Seoul|     1|      0|
|20.0|   Seoul|     0|      0|
+----+--------+------+-------+
only showing top 5 rows



In [ ]:
df_new.printSchema()

root
 |-- age: double (nullable = true)
 |-- province: string (nullable = true)
 |-- is_mal: integer (nullable = true)
 |-- is_dead: integer (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Imputer
from pyspark.ml.classification import RandomForestClassifier ,DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline

In [ ]:
df_train, df_test = df_new.randomSplit([0.8,0.2],seed=42)

In [ ]:
stringind = StringIndexer(inputCols=['province'],outputCols=['province_ind'])
onehot = OneHotEncoder(inputCols=['province_ind','is_mal'],outputCols=['province_hot','is_mal_hot'])
imputer = Imputer(inputCols=["age"], outputCols=["age_imput"]).setStrategy("mean")
vecAssemb = VectorAssembler(inputCols=['province_hot','is_mal_hot','age_imput'],outputCol='features')
Dtree = DecisionTreeClassifier(featuresCol='features',labelCol='is_dead')
RandomForest = RandomForestClassifier(featuresCol="features", labelCol="is_dead")

In [ ]:
pipeline1 = Pipeline(stages=[stringind, onehot, imputer, vecAssemb, RandomForest])

In [ ]:
model_RandomForest = pipeline1.fit(df_train)

In [ ]:
predict = model_RandomForest.transform(df_test)

evaluator = BinaryClassificationEvaluator(labelCol="is_dead")
auc = evaluator.evaluate(predict)

print("accuracy Binary:", auc)

evaluator = MulticlassClassificationEvaluator(labelCol="is_dead")
F1 = evaluator.evaluate(predict)
print("F1 Multi:", F1)

accuracy Binary: 0.9080872531884202
F1 Multi: 0.8754673875472279
